## Exercise 2

Exercise 2 includes two files:
 * wine.py include all the solution
 * test_wine.py unit test for wine.py

In [1]:
from owlready2 import get_ontology
from owlready2 import sync_reasoner

#### 1. List all grape growing regions (in the ontology)
* before get the result, runing the HermiT to reason the ontology
* return list

In [2]:
# List all grape growing regions (in the ontology)
def get_all_regions():
    '''
    Get all grape growing regions
    :return: List of grape growing regions
    '''
    onto = get_ontology('file://./wine.owl').load()
    with onto:
        sync_reasoner()
        return [x.name for x in onto.region.instances()]

#### 2. List all varietals (in the ontology)
* before get the result, runing the HermiT to reason the ontology
* tranverse whole branch to get all subclasses
* include 'also_called' value
* return list


In [3]:
def get_all_varietals():
    '''
    Get all grape varietals, include also_called
    :return: List of grape varietals
    '''
    onto = get_ontology('file://./wine.owl').load()
    with onto:
        sync_reasoner()
        subs = onto.varietal.descendants()
        subs.remove(onto.varietal)
        res = [x.name for x in subs]
        for c in subs:
            if len(c.also_called) > 0:
                res += c.also_called
        return res

#### 3. List all types (classes) of wine (in the ontology)
* before get the result, runing the HermiT to reason the ontology
* tranverse whole branch to get all subclasses
* return list

In [4]:
def get_all_types():
    '''
    List all types of wine
    :return: List of wine types
    '''
    onto = get_ontology('file://./wine.owl').load()
    with onto:
        sync_reasoner()
        subs = onto.color.descendants()
        subs.remove(onto.color)
        return [x.name for x in subs]

#### 4. Query for wine types and individual wines by: colour, varietal, region
* before get the result, runing the HermiT to reason the ontology
* three parameters, default value = None
* caputure instances of the entities
* remove duplicates

In [5]:
# Query for wine types and individual wines by: colour, varietal, region
def query(color=None, varietal=None, region=None):
    '''
    :param color: wine color
    :param varietal: grape varietal
    :param region: region
    :return: types and individuals by given colour, varietal and region
    '''
    onto = get_ontology('file://./wine.owl').load()
    with onto:
        sync_reasoner()
        wines = onto.wine.descendants()
        wines.remove(onto.wine)
        res = list(wines.copy())
        for wine in wines:
            if color:
                wine_c = [x.name.lower() for x in wine.has_color]
                if color.lower() not in wine_c:
                    res.remove(wine)
                    continue
            if varietal:
                wine_v = [x.name.lower() for x in wine.made_from]
                if varietal.lower() not in wine_v:
                    res.remove(wine)
                    continue
            if region:
                wine_r = [x.name.lower() for x in wine.grown_in]
                if region.lower() not in wine_r:
                    res.remove(wine)
                    continue

        for i in res:
            try:
                if len(i.instances()) > 0:
                    inst = i.instances()
                    res += inst
            except:
                pass
        return list(set([x.name for x in res]))

### Test file
1. get_all_regions()
2. get_all_varietals()
3. get_all_types()
4. query() # all None values
5. query(varietal='Canaiolo', region='Chianti') # with varietal & region values
6. query(color='red', region='Piedmont') # with color & region
7. query(color='red') # color only
8. query(color='blue') # unmatch